# Language Model

In [1]:
import random

In [2]:
filename = '/Users/zhuxiang/Documents/NLP/datasource-master/datasource-master/export_sql_1558435/sqlResult_1558435.csv'

In [3]:
import pandas as pd

In [4]:
content = pd.read_csv(filename, encoding = 'gb18030')

In [5]:
content.head()

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [6]:
articles = content['content'].tolist()

In [7]:
len(articles)

89611

In [8]:
import re
import pandas as pd

In [9]:
def token(string):
    return re.findall('\w+', string)

In [10]:
articles_clean = [''.join(token(str(a))) for a in articles]

In [11]:
len(articles_clean)

89611

In [12]:
with open('article_clean.txt','w') as f:
    for a in articles_clean:
        f.write(a + '\n')

In [13]:
import jieba
from collections import Counter

In [14]:
def cut(string): return list(jieba.cut(string))

In [15]:
TOKEN = []
for i, line in enumerate((open('article_clean.txt'))):
    if i % 1000 == 0 : print(i)
    
    if i > 89600: break
    
    TOKEN += cut(line)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\zhuxiang\AppData\Local\Temp\jieba.cache


0


Loading model cost 1.586 seconds.
Prefix dict has been built succesfully.


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000


In [16]:
len(TOKEN)

17616321

In [17]:
words_count = Counter(TOKEN)

In [18]:
def prob_1(word):
    return words_count[word] / len(TOKEN)

In [31]:
TOKEN = [str(t) for t in TOKEN]

In [32]:
prob_1('客户')

0.0001122822409968574

In [33]:
words_count.most_common(5)

[('的', 703628), ('n', 381962), ('在', 263558), ('月', 189306), ('日', 166281)]

In [34]:
TOKEN[:5]

['此外', '自', '本周', '6', '月']

In [37]:
TOKEN_2_GRAM = [''.join(TOKEN[i:i + 2]) for i in range(len(TOKEN[:-2]))]

In [38]:
TOKEN_2_GRAM[:5]

['此外自', '自本周', '本周6', '6月', '月12']

In [39]:
words_count_2 = Counter(TOKEN_2_GRAM)

In [40]:
def prob_1(word): return words_count[word] / len(TOKEN)

In [42]:
def prob_2(word1, word2):
    if word1 + word2 in words_count_2: return words_count_2[word1+word2] / len(TOKEN_2_GRAM)
    else:
        return 1 / len(TOKEN_2_GRAM)

In [43]:
prob_2('我们','在')

3.053986476970586e-05

In [44]:
def get_probablity(sentence):
    words = cut(sentence)
    
    sentence_pro = 1
    
    for i, word in enumerate(words[:-1]):
        next_ = words[i+1]
        
        probablity = prob_2(word, next_)
        
        sentence_pro *= probablity
        
    return sentence_pro

In [45]:
get_probablity('小明今天抽到一台苹果手机')

2.6523857366450873e-35

In [46]:
def get_best_sentence_as_you_want(grammar,target, n):
    result = []
    
    for s in [generate(gram = example_grammar, target = str(target)) for i in range(n)]:
        result.append((s, get_probablity(s)))
    
    print(sorted(result, key = lambda x: x[1]))

In [ ]:
get_best_sentence_as_you_want()